# Project Imports

In [1]:
from project_utils import config_paths
config_paths('omri', 'my_env') # Change to your name and your environment folder!
from project_utils import read_images, mean_iou, rle_encoding, prob_to_rles, predict_results, create_submission_file
from skimage.transform import resize
from skimage.morphology import label
from keras.callbacks import EarlyStopping, ModelCheckpoint
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import time

/home/omri/my_gpu/.env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
main_tic = time.time()

# Model Imports

In [3]:
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import tensorflow as tf

# Constants

In [4]:
TRAIN_PATH = '../../stage1_train/'
TEST_PATH = '../../stage1_test/'
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3

# Read the data

In [5]:
X_train, Y_train, X_test, sizes_test  = read_images(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS, TRAIN_PATH, TEST_PATH)

Getting and resizing train images and masks ... 


100%|██████████| 650/650 [01:34<00:00,  6.89it/s]

Getting and resizing test images ... 



100%|██████████| 65/65 [00:00<00:00, 95.76it/s] 

Done!


# Build your model here:

In [6]:
inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.1) (c9)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[mean_iou])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 128, 128, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 16) 448         lambda_1[0][0]                   
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 128, 128, 16) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (

# Train

In [7]:
tic = time.time()
model_name = 'lalala' 
model_name = model_name + '.h5'
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint(model_name, verbose=1, save_best_only=True)
results = model.fit(X_train, Y_train, validation_split=0.1, batch_size=16, epochs=50, 
                    callbacks=[earlystopper, checkpointer])

toc = time.time() - tic
print('Training time...', toc/60)

Train on 585 samples, validate on 65 samples
Epoch 1/50
585/585 [==============================] - 10s 17ms/step - loss: 0.4282 - mean_iou: 0.4176 - val_loss: 0.2699 - val_mean_iou: 0.4371

Epoch 00001: val_loss improved from inf to 0.26989, saving model to lalala.h5
Epoch 2/50
585/585 [==============================] - 7s 11ms/step - loss: 0.2260 - mean_iou: 0.4740 - val_loss: 0.1853 - val_mean_iou: 0.5128

Epoch 00002: val_loss improved from 0.26989 to 0.18528, saving model to lalala.h5
Epoch 3/50
585/585 [==============================] - 6s 11ms/step - loss: 0.1674 - mean_iou: 0.5456 - val_loss: 0.1648 - val_mean_iou: 0.5748

Epoch 00003: val_loss improved from 0.18528 to 0.16477, saving model to lalala.h5
Epoch 4/50
585/585 [==============================] - 7s 11ms/step - loss: 0.1591 - mean_iou: 0.5957 - val_loss: 0.1500 - val_mean_iou: 0.6132

Epoch 00004: val_loss improved from 0.16477 to 0.15001, saving model to lalala.h5
Epoch 5/50
585/585 [==============================] - 

# Predict

In [8]:
preds_test_upsampled = predict_results(model_name, X_train, X_test, sizes_test)

65/65 [==============================] - 0s 2ms/step


# Create a submission file

In [9]:
filename = 'lalalala'
test_ids = os.listdir(TEST_PATH)
sub = create_submission_file(preds_test_upsampled, test_ids, filename)

In [10]:
sub.head()

,ImageId,EncodedPixels
0,0999dab07b11bc85fb8464fc36c947fbd8b5d6ec498173...,50346 1 50864 4 51383 6 51902 7 51919 6 52421 ...
1,0999dab07b11bc85fb8464fc36c947fbd8b5d6ec498173...,121968 1 122486 3 123005 3 123524 4 124043 5 1...
2,0999dab07b11bc85fb8464fc36c947fbd8b5d6ec498173...,24416 8 24933 13 25450 17 25967 20 26485 21 27...
3,0999dab07b11bc85fb8464fc36c947fbd8b5d6ec498173...,118366 4 118883 8 119401 10 119919 12 120438 1...
4,0999dab07b11bc85fb8464fc36c947fbd8b5d6ec498173...,6279 1 6796 7 7313 12 7831 14 8349 16 8868 17 ...


In [11]:
print('Total runtime...', (time.time() - main_tic)/60)

Total runtime... 5.365939307212829
